# AWS Assignment – Colab Workbook
Organized answers with ready-to-run commands and code snippets. Replace placeholder values (**ALL_CAPS**) with your own (e.g., bucket names, regions, ARNs).

> Tip: In Colab, you can execute shell commands by prefixing with `!` and install packages with `pip`.


<a id="setup"></a>

## Setup: Variables & AWS CLI

Set your variables here. In Colab, you can authenticate to AWS by configuring credentials (`aws configure`) or using environment variables/roles (when running on an EC2/SageMaker/Cloud9 instance).

<a id="q1"></a>

## Q1: Explain the difference between AWS Regions, Availability Zones, and Edge Locations. Why is this important for data analysis and latency-sensitive applications?

- **Region**: A separate geographic area (e.g., `ap-south-1` in Mumbai) consisting of multiple isolated locations (AZs). Data residency, regulatory control, and service availability vary by region.
- **Availability Zone (AZ)**: One or more discrete data centers within a region with independent power/network. Deploying across multiple AZs improves **high availability** and **fault tolerance**.
- **Edge Location**: Part of the **Amazon CloudFront** and **Global Accelerator** network, placed near end users to cache content or accelerate traffic, reducing **latency**.

**Why it matters**:
- Analytics jobs near data sources (same **Region**) avoid cross-region data transfer cost/latency.
- Distributing compute across **AZs** reduces downtime for pipelines and streaming consumers.
- **Edge** caching/ingestion lowers latency for dashboards and streaming ingestion from global producers.


<a id="q2"></a>

## Q2: Using the AWS CLI, list all available AWS regions. Share the command and output.

In [ ]:
# If AWS CLI is installed & configured, run:
# !aws ec2 describe-regions --all-regions --query 'Regions[].{Region:RegionName, OptInStatus:OptInStatus}' --output table

# Alternate (names only):
# !aws ec2 describe-regions --all-regions --query 'Regions[].RegionName' --output text


<a id="q3"></a>

## Q3: Create a new IAM user with least privilege access to Amazon S3. Share attached policies.

**Principles**: grant only the S3 permissions required (e.g., read/write to a specific bucket path). Avoid `*` where possible.

**Steps (CLI)**:

In [ ]:
# Create user (programmatic access via access key)
# !aws iam create-user --user-name s3-data-user

# Attach an inline policy allowing least-privilege S3 access to a single bucket/prefix
import json
least_priv_policy = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "ListBucketAtPrefix",
      "Effect": "Allow",
      "Action": ["s3:ListBucket"],
      "Resource": [f"arn:aws:s3:::{BUCKET}"],
      "Condition": {"StringLike": {"s3:prefix": [f"{DATA_PREFIX}/*"]}}
    },
    {
      "Sid": "RWObjectsUnderPrefix",
      "Effect": "Allow",
      "Action": ["s3:GetObject","s3:PutObject","s3:DeleteObject","s3:AbortMultipartUpload"],
      "Resource": [f"arn:aws:s3:::{BUCKET}/{DATA_PREFIX}/*"]
    }
  ]
}
print(json.dumps(least_priv_policy, indent=2))

# Put the inline policy (save first):
# with open("least_priv_policy.json","w") as f: json.dump(least_priv_policy, f)
# !aws iam put-user-policy --user-name s3-data-user --policy-name S3PrefixLeastPrivilege --policy-document file://least_priv_policy.json

# (Optional) Create access keys for the user (handle securely, never commit to source control!)
# !aws iam create-access-key --user-name s3-data-user


<a id="q4"></a>

## Q4: Compare S3 storage classes (Standard, Intelligent-Tiering, Glacier). When to use each in analytics?

| Storage Class | Durability/Availability | Cost | Use in Analytics |
|---|---|---|---|
| **Standard** | 11x9s / high | Higher | Active datasets, staging/landing zones, frequent reads/writes |
| **Intelligent‑Tiering** | 11x9s / high | Optimizes | Unpredictable access; auto‑moves between frequent/infrequent tiers without operational work |
| **Glacier (Flexible/Deep Archive)** | 11x9s / variable retrieval times | Lowest at rest | Long‑term retention, rarely accessed raw data, audit archives; retrieve before batch analysis |


<a id="q5"></a>

## Q5: Create an S3 bucket and upload a sample dataset; enable versioning and show versions

In [ ]:
# Create bucket (name must be globally unique)
# !aws s3 mb s3://$BUCKET --region $REGION

# Enable versioning
# !aws s3api put-bucket-versioning --bucket $BUCKET --versioning-configuration Status=Enabled

# Upload sample dataset
sample_csv = "name,age\nAsha,29\nRavi,33\nZoya,41\n"
with open(TEST_FILE, "w") as f: f.write(sample_csv)
print("Wrote local sample CSV:", TEST_FILE)

# First version
# !aws s3 cp $TEST_FILE s3://$BUCKET/$DATA_PREFIX/$TEST_FILE

# Modify & upload again to create a second version
sample_csv_v2 = "name,age\nAsha,30\nRavi,33\nZoya,41\n"
with open(TEST_FILE, "w") as f: f.write(sample_csv_v2)
print("Updated CSV to create v2.")

# Second version
# !aws s3 cp $TEST_FILE s3://$BUCKET/$DATA_PREFIX/$TEST_FILE

# List object versions
# !aws s3api list-object-versions --bucket $BUCKET --prefix $DATA_PREFIX/$TEST_FILE --query 'Versions[].{Id:VersionId,LastModified:LastModified,IsLatest:IsLatest}' --output table


<a id="q6"></a>

## Q6: Lifecycle policy (move to Glacier after 30 days, delete after 90)

In [ ]:
import json
lifecycle_policy = {
  "Rules": [
    {
      "ID": "ToGlacierThenExpire",
      "Filter": {"Prefix": f"{DATA_PREFIX}/"},
      "Status": "Enabled",
      "Transitions": [
        {"Days": 30, "StorageClass": "GLACIER"}
      ],
      "Expiration": {"Days": 90}
    }
  ]
}
print(json.dumps(lifecycle_policy, indent=2))

# with open("lifecycle.json","w") as f: json.dump(lifecycle_policy, f)
# !aws s3api put-bucket-lifecycle-configuration --bucket $BUCKET --lifecycle-configuration file://lifecycle.json


<a id="q7"></a>

## Q7: Compare RDS, DynamoDB, and Redshift with one data-pipeline use case each

- **Amazon RDS (relational)**: OLTP, transactional integrity (ACID), SQL joins. *Use case*: store cleaned, modeled app data used by downstream analytics and operational dashboards.
- **Amazon DynamoDB (NoSQL key‑value/Document)**: massive scale, single‑digit ms latency, serverless ops. *Use case*: capture clickstream/session events with partition keys for hot ingestion; later ETL to S3.
- **Amazon Redshift (MPP analytical columnar)**: petabyte‑scale, columnar storage, parallel queries. *Use case*: star‑schema warehouse powering BI with complex aggregations and joins.


<a id="q8"></a>

## Q8: Create a DynamoDB table and insert 3 records manually. Lambda adds records on S3 uploads.

In [ ]:
# Create table (Partition key: pk, Sort key: sk)
# !aws dynamodb create-table --table-name $DDB_TABLE \
#   --attribute-definitions AttributeName=pk,AttributeType=S AttributeName=sk,AttributeType=S \
#   --key-schema AttributeName=pk,KeyType=HASH AttributeName=sk,KeyType=RANGE \
#   --billing-mode PAY_PER_REQUEST

# Insert 3 records
# !aws dynamodb put-item --table-name $DDB_TABLE --item '{"pk":{"S":"user#1"},"sk":{"S":"evt#001"},"info":{"S":"seed"}}'
# !aws dynamodb put-item --table-name $DDB_TABLE --item '{"pk":{"S":"user#2"},"sk":{"S":"evt#001"},"info":{"S":"seed"}}'
# !aws dynamodb put-item --table-name $DDB_TABLE --item '{"pk":{"S":"user#3"},"sk":{"S":"evt#001"},"info":{"S":"seed"}}'

lambda_code = (
"import json, boto3, os, urllib.parse\n"
"dynamo = boto3.resource(\"dynamodb\")\n"
"table = dynamo.Table(os.environ[\"TABLE_NAME\"])\n\n"
"def handler(event, context):\n"
"    for record in event.get(\"Records\", []):\n"
"        s3 = record.get(\"s3\", {})\n"
"        bucket = s3.get(\"bucket\", {}).get(\"name\")\n"
"        key = urllib.parse.unquote_plus(s3.get(\"object\", {}).get(\"key\", \"\"))\n"
"        size = s3.get(\"object\", {}).get(\"size\", 0)\n"
"        table.put_item(Item={\n"
"            'pk': f's3#{bucket}',\n"
"            'sk': f'object#{key}',\n"
"            'size': int(size),\n"
"            'ts': record.get('eventTime')\n"
"        })\n"
"    return {'status': 'ok'}\n"
)
print(lambda_code)

# Create the Lambda, add S3 trigger, and environment variable TABLE_NAME=$DDB_TABLE.
# Grant IAM permissions: dynamodb:PutItem on the table.


<a id="q9"></a>

## Q9: What is serverless computing? Pros & cons of Lambda for data pipelines

**Serverless** means you focus on code; the provider manages servers, scaling, and patching. You pay per use.

**Pros**:
- Auto‑scaling and high availability by default
- Pay‑per‑invocation; cost‑efficient for spiky workloads
- Easy event integrations (S3, SQS, EventBridge, Kinesis)

**Cons**:
- Cold starts (latency) for some runtimes/VPC configs
- Execution time/memory/ephemeral storage limits
- Stateful/long‑running tasks need orchestration (e.g., Step Functions, Glue, ECS)


<a id="q10"></a>

## Q10: Lambda function triggered by S3 that logs file name, size, timestamp to CloudWatch

In [ ]:
lambda_logger = (
"import json, logging, urllib.parse\n\n"
"logger = logging.getLogger()\n"
"logger.setLevel('INFO')\n\n"
"def handler(event, context):\n"
"    for record in event.get('Records', []):\n"
"        s3 = record.get('s3', {})\n"
"        bucket = s3.get('bucket', {}).get('name')\n"
"        key = urllib.parse.unquote_plus(s3.get('object', {}).get('key', ''))\n"
"        size = s3.get('object', {}).get('size', 0)\n"
"        ts = record.get('eventTime')\n"
"        logger.info({'bucket': bucket, 'key': key, 'size': size, 'timestamp': ts})\n"
"    return {'status':'ok'}\n"
)
print(lambda_logger)


<a id="q11"></a>

## Q11: Use AWS Glue to crawl S3 data, create a Data Catalog table, and run a job to convert CSV → Parquet
**Steps**:
1. Create a **Crawler** pointing to `s3://BUCKET/datasets/sample/` with an IAM role that can read S3 and write to the **Glue Data Catalog** (database like `analytics_db`).
2. Run the crawler to create/update a table (e.g., `analytics_db.people`).
3. Create a **Glue Spark** job with the script below to read CSV and write Parquet to `s3://BUCKET/output/people_parquet/`.


In [ ]:
glue_job_script = (
"import sys\n"
"from awsglue.transforms import *\n"
"from awsglue.utils import getResolvedOptions\n"
"from pyspark.context import SparkContext\n"
"from awsglue.context import GlueContext\n"
"from awsglue.job import Job\n\n"
"args = getResolvedOptions(sys.argv, ['JOB_NAME','SOURCE_S3','OUTPUT_S3'])\n"
"sc = SparkContext()\n"
"glueContext = GlueContext(sc)\n"
"spark = glueContext.spark_session\n"
"job = Job(glueContext); job.init(args['JOB_NAME'], args)\n\n"
"df = spark.read.option('header','true').csv(args['SOURCE_S3'])\n"
"df.write.mode('overwrite').parquet(args['OUTPUT_S3'])\n\n"
"job.commit()\n"
)
print(glue_job_script)


<a id="q12"></a>

## Q12: Difference between Kinesis Data Streams, Kinesis Data Firehose, and Kinesis Data Analytics + examples

- **Kinesis Data Streams (KDS)**: real‑time, low‑latency streaming service with shards and consumer apps. *Example*: custom consumers (Lambda/Flink) process clickstreams within seconds.
- **Kinesis Data Firehose (KDF)**: fully managed delivery from sources to destinations (S3, Redshift, OpenSearch) with minimal config; buffering/format conversion built‑in. *Example*: deliver IoT events to S3 in Parquet every 1–5 minutes.
- **Kinesis Data Analytics (KDA)**: run SQL or Apache Flink apps on streaming data. *Example*: continuous SQL aggregations on KDS, pushing results to Firehose or Lambda.


<a id="q13"></a>

## Q13: What is columnar storage and how does it benefit Redshift performance?

Columnar storage saves values column‑by‑column instead of row‑by‑row. Benefits for analytics:
- **I/O reduction**: scan only needed columns for SELECTs.
- **Compression**: similar adjacent values compress well, reducing disk/memory usage.
- **Vectorized processing**: efficient CPU utilization for aggregates/scans.


<a id="q14"></a>

## Q14: Load a CSV from S3 into Redshift using COPY (schema, command, sample query)

In [ ]:
redshift_schema_sql = """
CREATE TABLE IF NOT EXISTS public.people (
  name VARCHAR(50),
  age  INTEGER
);
"""
copy_cmd = """
COPY public.people
FROM '{S3_SOURCE}'
IAM_ROLE '{REDSHIFT_IAM_ROLE_ARN}'
CSV IGNOREHEADER 1 REGION '{REGION}';
"""
sample_query = "SELECT COUNT(*) AS rows_loaded, AVG(age)::numeric(10,2) AS avg_age FROM public.people;"
print("-- Schema DDL --\n", redshift_schema_sql)
print("\n-- COPY command (replace placeholders in braces) --\n", copy_cmd)
print("\n-- Sample query --\n", sample_query)
# Replace {S3_SOURCE}, {REDSHIFT_IAM_ROLE_ARN}, and {REGION} with your values before executing in Redshift.


<a id="q15"></a>

## Q15: Role of the Glue Data Catalog in Athena; how schema‑on‑read works

- **Glue Data Catalog** stores table/partition metadata (locations, schemas, SerDes) for data in S3.
- **Athena** uses this metadata to interpret files at query time (**schema‑on‑read**) rather than enforcing schema during ingestion. You can change the schema independently, and the same data can be queried by different tools without rewriting files.


<a id="q16"></a>

## Q16: Create an Athena table from S3 data using Glue Catalog; run a query

In [ ]:
athena_ddl = """
CREATE DATABASE IF NOT EXISTS analytics_db;
CREATE EXTERNAL TABLE IF NOT EXISTS analytics_db.people (
  name string,
  age  int
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES ('separatorChar'=',','quoteChar'='\"','escapeChar'='\\')
LOCATION '{S3_SOURCE}'
TBLPROPERTIES ('has_encrypted_data'='false');
"""
athena_query = "SELECT name, age FROM analytics_db.people WHERE age >= 30 ORDER BY age DESC;"
print("-- Athena DDL (replace {S3_SOURCE}) --\n", athena_ddl)
print("\n-- Sample Query --\n", athena_query)
# In Athena console, set a Query Result Location in S3 before running.


<a id="q17"></a>

## Q17: How Amazon QuickSight supports BI in a serverless data architecture; SPICE & embedded dashboards

- **Serverless BI**: QuickSight connects to Athena/Redshift/S3; no servers to manage for the BI layer.
- **SPICE**: QuickSight's in‑memory engine for fast, interactive analysis; you can import data into SPICE to accelerate dashboards and control refresh schedules.
- **Embedded dashboards**: securely embed dashboards into apps/portals with row‑level security and AWS IAM/Federation integration.


<a id="q18"></a>

## Q18: Connect QuickSight to Athena or Redshift; build a dashboard (calc field + filter)

**Steps**:
1. In QuickSight, create a **Data Source** to **Athena** (or **Redshift**). Grant QuickSight the required IAM permissions.
2. Choose your database/table (e.g., `analytics_db.people`) and **import to SPICE** for speed.
3. Create a dataset; add a **calculated field** (e.g., `is_senior = ifelse({age} >= 60, 1, 0)`).
4. Build visuals (e.g., table + KPI); add a **Filter** on `age` or a date field.
5. Publish the analysis as a **dashboard**. (Optional) Use the **embedding** options for apps/portals.


<a id="q19"></a>

## Q19: CloudWatch vs CloudTrail in a data analytics pipeline

- **CloudWatch**: metrics, logs, alarms, dashboards. Use for pipeline health (Lambda duration/errors, Glue job logs, Kinesis throughput), alerting on failures or lag.
- **CloudTrail**: governance/auditing of **API activity** across AWS accounts. Use to trace who changed an S3 bucket policy or who ran a Glue job, and to investigate incidents.


<a id="q20"></a>

## Q20: End‑to‑end data analytics pipeline (example)

**Ingestion**: App/IoT → **Kinesis Firehose** → **S3 landing**  
**Processing**: S3 event → **Lambda** for light validation; batch ETL in **Glue** (CSV→Parquet, partitioned by date) → **S3 curated**  
**Catalog/Query**: **Glue Data Catalog** tables over curated S3; **Athena** for ad‑hoc SQL; **Redshift** for complex joins/BI marts  
**Visualization**: **QuickSight** (SPICE for speed, embedded dashboard)  
**Ops**: **CloudWatch** for logs/alarms/dashboards; **CloudTrail** for audit; **IAM** least privilege

**Why these choices**: serverless, scalable, cost‑efficient; Parquet + partitioning minimizes scan costs; separation of landing/curated zones improves governance; Glue/Athena/Redshift cover ELT and analytics at multiple scales.
